# 5.5 读写文件
- **目录**
  - 5.5.1 加载和保存张量
  - 5.5.2 加载和保存模型参数


- 到目前为止，我们讨论了如何处理数据，以及如何构建、训练和测试深度学习模型。
- 有时我们希望保存训练的模型，以备将来在各种环境中使用（比如在部署中进行预测）。
- 此外，当运行一个耗时较长的训练过程时，最佳的做法是**定期保存中间结果**，以确保在服务器电源被不小心断掉时，我们不会损失几天的计算结果。
- 因此掌握加载和存储权重向量和整个模型的技巧很重要。

## 5.5.1 加载和保存张量

- 对于单个张量，可以直接调用`load`和`save`函数分别读写它们。
- 这两个函数都要求提供一个文件名称。
- `save`要求将要保存的变量作为输入。

In [4]:
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file')

- 现在可以将存储在文件中的数据读回内存。


In [5]:
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

- 可以存储一个张量列表，然后把它们读回内存。


In [6]:
## 操作还是很简便的
y = torch.zeros(4)
torch.save([x, y],'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

- 可以写入或读取从字符串映射到张量的字典。
- 当需要读取或写入模型中的所有权重时，该方法很方便。


In [7]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

## 5.5.2 加载和保存模型参数

- 深度学习框架提供了内置函数来保存和加载整个网络。
- 需要注意的一个重要细节是，这将保存模型的参数而不是保存整个模型。
    - 例如，对于某个3层多层感知机，则需要单独指定架构。
- 因为模型本身可以包含任意代码，所以模型本身难以序列化。
- 因此，为了恢复模型，我们需要用代码生成架构，然后从磁盘加载参数。

In [8]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        '''
          层的名字可以任意命名，当然要有一定意义，便于理解。
          比如此处在原代码的基础上，增加了一个hidden2层，原hidden改成hidden1
        '''
        self.hidden1 = nn.Linear(20, 256)
        self.hidden2 = nn.Linear(256, 128)##此层是我加的
        self.output = nn.Linear(128, 10)

    def forward(self, x):
        ##原代码
        #return self.output(F.relu(self.hidden(x)))
        
        ## 对原来的forward函数进行了修改
        ## 注意此处的嵌套调用方式
        return self.output(F.relu(self.hidden2(F.relu(self.hidden1(x)))))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)
Y

tensor([[ 0.0692, -0.0820,  0.0069,  0.1195, -0.1927, -0.2309,  0.0661,  0.0667,
          0.0261, -0.0405],
        [ 0.0408,  0.0498,  0.0231,  0.0177, -0.0366, -0.0824, -0.0893,  0.0744,
         -0.0338,  0.0079]], grad_fn=<AddmmBackward0>)

- 将模型的参数存储在一个叫做“mlp.params”的文件中。


In [9]:
## 模型的参数保存在state_dict中
torch.save(net.state_dict(), 'mlp.params')

- 为了恢复模型，实例化了原始多层感知机模型的一个备份。
- 不需要随机初始化模型参数，而是**直接读取文件中存储的参数。**


In [10]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden1): Linear(in_features=20, out_features=256, bias=True)
  (hidden2): Linear(in_features=256, out_features=128, bias=True)
  (output): Linear(in_features=128, out_features=10, bias=True)
)

In [11]:
# 克隆模型的参数
clone.state_dict()

OrderedDict([('hidden1.weight',
              tensor([[ 0.1710, -0.0664, -0.0360,  ...,  0.1741, -0.0216,  0.1797],
                      [ 0.1008, -0.1274,  0.1556,  ...,  0.0724, -0.2218, -0.0725],
                      [ 0.1780,  0.0977, -0.1743,  ...,  0.1828,  0.0696, -0.0862],
                      ...,
                      [ 0.0681, -0.0513,  0.1119,  ..., -0.1003, -0.0724,  0.0553],
                      [ 0.0176,  0.1683,  0.2015,  ..., -0.2133, -0.0777,  0.1066],
                      [-0.0421, -0.0341, -0.1183,  ...,  0.0671,  0.2195, -0.1041]])),
             ('hidden1.bias',
              tensor([-0.1316,  0.0226, -0.0699, -0.1633, -0.1629,  0.0876,  0.0511,  0.0073,
                      -0.0837, -0.1180,  0.1689, -0.1043, -0.1351,  0.0022,  0.1146,  0.0863,
                      -0.0722,  0.1177, -0.1565, -0.1193,  0.1253, -0.0268, -0.1376,  0.2009,
                       0.0311, -0.1901,  0.0541,  0.0132, -0.0219,  0.0113, -0.0717,  0.0252,
                       0.064

In [12]:
## 模型的参数
net.state_dict()

OrderedDict([('hidden1.weight',
              tensor([[ 0.1710, -0.0664, -0.0360,  ...,  0.1741, -0.0216,  0.1797],
                      [ 0.1008, -0.1274,  0.1556,  ...,  0.0724, -0.2218, -0.0725],
                      [ 0.1780,  0.0977, -0.1743,  ...,  0.1828,  0.0696, -0.0862],
                      ...,
                      [ 0.0681, -0.0513,  0.1119,  ..., -0.1003, -0.0724,  0.0553],
                      [ 0.0176,  0.1683,  0.2015,  ..., -0.2133, -0.0777,  0.1066],
                      [-0.0421, -0.0341, -0.1183,  ...,  0.0671,  0.2195, -0.1041]])),
             ('hidden1.bias',
              tensor([-0.1316,  0.0226, -0.0699, -0.1633, -0.1629,  0.0876,  0.0511,  0.0073,
                      -0.0837, -0.1180,  0.1689, -0.1043, -0.1351,  0.0022,  0.1146,  0.0863,
                      -0.0722,  0.1177, -0.1565, -0.1193,  0.1253, -0.0268, -0.1376,  0.2009,
                       0.0311, -0.1901,  0.0541,  0.0132, -0.0219,  0.0113, -0.0717,  0.0252,
                       0.064

In [13]:
net ## 原模型有两层，一层是隐藏层，一层是输出层，此处修改成了3层

MLP(
  (hidden1): Linear(in_features=20, out_features=256, bias=True)
  (hidden2): Linear(in_features=256, out_features=128, bias=True)
  (output): Linear(in_features=128, out_features=10, bias=True)
)

- 由于两个实例具有相同的模型参数，在输入相同的`X`时，两个实例的计算结果应该相同。

In [14]:
## clone是通过加载参数文件后生成的模型
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

## 小结

* `save`和`load`函数可用于张量对象的文件读写。
* 我们可以通过参数字典保存和加载网络的全部参数。
* 保存架构必须在代码中完成，而不是在参数中完成。